<a href="https://colab.research.google.com/github/poovendhiranr/Usecase/blob/main/movie_logi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

import pandas as pd
import numpy as np
import unicodedata
from bs4 import BeautifulSoup
import re
import tqdm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
dataset = pd.read_csv("/content/movie_reviews.csv.bz2")
dataset.info()
dataset.head()

def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
  return stripped_text

def remove_accented_characters(text):
  text = unicodedata.normalize("NFKD",text).encode("ascii",'ignore').decode('utf-8','ignore')
  return text

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [16]:
reviews  = dataset['review'].values
sentiment = dataset['sentiment'].values
print(dataset.head(10))

print(reviews)

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive
["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me

In [18]:
import contractions
def pre_process_corpus(texts):
  preprocessed_text = []
  for doc in tqdm.tqdm(texts):
    doc = strip_html_tags(doc)
    #doc = doc.translate(doc.maketrans("\n\t\r",'   '))
    doc = doc.lower()
    doc = remove_accented_characters(doc)
    doc = contractions.fix(doc)
    doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,re.I|re.A)
    doc = re.sub(r' +',' ',doc)
    doc = doc.strip()
    #doc = "".join(doc)
    preprocessed_text.append(doc)
  return preprocessed_text

reviews  = dataset['review'].values
sentiment = dataset['sentiment'].values

train_reviews = reviews[:35000]
train_sentiment = sentiment[:35000]

test_reviews = reviews[35000:]
test_sentiment = sentiment[35000:]

norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

  2%|▏         | 796/35000 [00:00<00:19, 1763.38it/s]<ipython-input-15-bf5e3892e846>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 15000/15000 [00:08<00:00, 1671.43it/s]


In [22]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer(binary = False,ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_train_reviews)
cv_test_features = cv.transform(norm_test_reviews)

Tv = TfidfVectorizer(use_idf= True,ngram_range=(1,2))
Tv_train_features = Tv.fit_transform(norm_train_reviews)
Tv_test_features = Tv.transform(norm_test_reviews)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
lr = LogisticRegression(penalty = 'l2',max_iter=500,C=1)
lr.fit(cv_train_features,train_sentiment)
lr_bow_predictions = lr.predict(cv_test_features)
labels = ['negative','positive']
print(classification_report(test_sentiment,lr_bow_predictions))
print("hello")
pd.DataFrame(confusion_matrix(test_sentiment, lr_bow_predictions),index = labels,columns = labels)


              precision    recall  f1-score   support

    negative       0.91      0.90      0.90      7490
    positive       0.90      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000

hello


,negative,positive
negative,6749,741
positive,708,6802


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
lr = LogisticRegression(penalty = 'l2',max_iter=500,C=1)
lr.fit(Tv_train_features,train_sentiment)
lr_tfidf_predictions = lr.predict(Tv_test_features)
labels = ['negative','positive']
print(classification_report(test_sentiment,lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiment, lr_tfidf_predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.90      0.88      0.89      7490
    positive       0.89      0.91      0.90      7510

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



,negative,positive
negative,6606,884
positive,705,6805


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
rf.fit(Tv_train_features, train_sentiment)
rf_tfidf_predictions = rf.predict(Tv_test_features)
rf_accuracy = accuracy_score(test_sentiment, rf_tfidf_predictions)

labels = ['negative','positive']
print(classification_report(test_sentiment,rf_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiment, rf_tfidf_predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.83      0.86      0.85      7490
    positive       0.86      0.83      0.84      7510

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000



,negative,positive
negative,6472,1018
positive,1295,6215


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
rf.fit(cv_train_features, train_sentiment)
rf_tfidf_predictions = rf.predict(cv_test_features)
rf_accuracy = accuracy_score(test_sentiment, rf_tfidf_predictions)

labels = ['negative','positive']
print(classification_report(test_sentiment,rf_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiment, rf_tfidf_predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.86      0.86      0.86      7490
    positive       0.86      0.86      0.86      7510

    accuracy                           0.86     15000
   macro avg       0.86      0.86      0.86     15000
weighted avg       0.86      0.86      0.86     15000



,negative,positive
negative,6404,1086
positive,1051,6459
